# Adding Object Detection Predictions to a Voxel51 Dataset
This notebook will add predictions from an object detection model to the samples in a Voxel51 Dataset.

Adapted from: https://voxel51.com/docs/fiftyone/recipes/model_inference.html

In [8]:
model_path = '/tf/model-export/jsm-322images-test-model/image_tensor_saved_model/saved_model' # The path of the saved Object Detection model
dataset_name = "jsm-test-dataset" # Name of the Voxel51 Dataset to use
field_name = "predict_model" # Name of the field to store the predictions in
labelmap_file = '/tf/dataset-export/jsm-322images-test-model/label_map.pbtxt' # the location of the labelmap file to use
min_score = 0.5 # This is the minimum score for adding a prediction. This helps keep out bad predictions but it may need to be adjusted if your model is not that good yet.
# dimensions of images

In [2]:
#config
import fiftyone as fo
import os

dataset = fo.load_dataset(dataset_name)  

In [3]:
import io
import os
import scipy.misc
import numpy as np
import six
import time

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from object_detection.utils import label_map_util
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
# small function that preprocesses the images so that the model can read them in
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
%matplotlib inline




### Export full Voxel 51 Dataset to TF Record Format
Having the example image ready in the TF Record format

### Load saved model
Loading a saved objection detection model is a little weird. I found some info on it:
https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/inference_from_saved_model_tf2_colab.ipynb

In [4]:
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load(model_path)
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')


Importing a function (__inference_EfficientDet-D0_layer_call_and_return_conditional_losses_100122) with ops with custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_bifpn_layer_call_and_return_conditional_losses_121940) with ops with custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_EfficientDet-D0_layer_call_and_return_conditional_losses_96379) with ops with custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_EfficientDet-D0_layer_call_and_return_conditional_losses_92771) with ops with custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_bifpn_layer_call_and_return_conditional_losses_60229) with ops with custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_bifpn_layer_call_and_return_conditional_losses_120320) with ops with custom gradients. Will likely fail if a 

### Load the LabelMap file

In [6]:
label_map = label_map_util.load_labelmap(labelmap_file)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=100)
category_index = label_map_util.create_category_index(categories)

def findClassName(class_id):
    return category_index[class_id]["name"]

### Add predictions
Itterate through all the samples, run them through the model and add the predictions to the sample

In [ ]:
view = dataset.shuffle() # Adjust the view as needed

start_time = time.time()
for sample in view.select_fields("filepath"):


    img = load_img(sample.filepath)
    img_array = img_to_array(img)
    input_tensor = np.expand_dims(img_array, 0)
    detections = detect_fn(input_tensor)
    exportDetections = []
    for i, detectScore in enumerate(detections['detection_scores'][0]):
        if detectScore > min_score:
            print("\t- {}: {}".format(findClassName(int(detections['detection_classes'][0][i])), detections['detection_scores'][0][i]))

            label = findClassName(int(detections['detection_classes'][0][i]))
            confidence = detections['detection_scores'][0][i]
            # TF Obj Detect bounding boxes are: [ymin, xmin, ymax, xmax]
            
            # For Voxel 51 - Bounding box coordinates should be relative values
            # in [0, 1] in the following format:
            # [top-left-x, top-left-y, width, height]
            x1 = detections['detection_boxes'][0][i][1]
            y1 = detections['detection_boxes'][0][i][0]
            x2 = detections['detection_boxes'][0][i][3]
            y2 = detections['detection_boxes'][0][i][2]
            w = x2 - x1
            h = y2 - y1
            bbox = [x1, y1, w, h]

            exportDetections.append( fo.Detection(label=label, bounding_box=bbox, confidence=confidence))
    

    # Store detections in a field name of your choice
    sample[field_name] = fo.Detections(detections=exportDetections)
    sample.save()    
end_time = time.time()
print("Total time: {}".format(end_time-start_time))

Load: 0.051068782806396484 Detection: 0.07239270210266113 Save: 0.10973834991455078 Total: 0.16080713272094727
Found 0 Planes, took 0.16080713272094727 seconds
Load: 0.05016016960144043 Detection: 0.07396674156188965 Save: 0.11144042015075684 Total: 0.16160058975219727
Found 0 Planes, took 0.16160058975219727 seconds
	- plane: 0.9997361302375793
Load: 0.044095516204833984 Detection: 0.07365655899047852 Save: 0.11668634414672852 Total: 0.1607818603515625
Found 1 Planes, took 0.1607818603515625 seconds
	- plane: 0.9997585415840149
Load: 0.04284501075744629 Detection: 0.07271170616149902 Save: 0.11941742897033691 Total: 0.1622624397277832
Found 1 Planes, took 0.1622624397277832 seconds
	- plane: 0.9999451637268066
Load: 0.04443764686584473 Detection: 0.07236337661743164 Save: 0.11578559875488281 Total: 0.16022324562072754
Found 1 Planes, took 0.16022324562072754 seconds
	- plane: 0.9988283514976501
Load: 0.04510807991027832 Detection: 0.07308793067932129 Save: 0.11695575714111328 Total: 0

	- plane: 0.999953031539917
Load: 0.052117109298706055 Detection: 0.07275700569152832 Save: 0.11455345153808594 Total: 0.166670560836792
Found 1 Planes, took 0.166670560836792 seconds
	- plane: 0.9964632391929626
Load: 0.04736733436584473 Detection: 0.0724949836730957 Save: 0.11438298225402832 Total: 0.16175031661987305
Found 1 Planes, took 0.16175031661987305 seconds
Load: 0.04361438751220703 Detection: 0.07288384437561035 Save: 0.10907387733459473 Total: 0.15268826484680176
Found 0 Planes, took 0.15268826484680176 seconds
	- plane: 0.9989553689956665
Load: 0.044995784759521484 Detection: 0.07309126853942871 Save: 0.11711525917053223 Total: 0.1621110439300537
Found 1 Planes, took 0.1621110439300537 seconds
Load: 0.04322481155395508 Detection: 0.07182812690734863 Save: 0.10954904556274414 Total: 0.15277385711669922
Found 0 Planes, took 0.15277385711669922 seconds
	- plane: 0.9997206330299377
	- plane: 0.7269232273101807
Load: 0.04835247993469238 Detection: 0.07241940498352051 Save: 0.1

	- plane: 0.9990912675857544
Load: 0.04363441467285156 Detection: 0.07315492630004883 Save: 0.11517667770385742 Total: 0.15881109237670898
Found 1 Planes, took 0.15881109237670898 seconds
	- plane: 0.9999798536300659
Load: 0.04503369331359863 Detection: 0.0729227066040039 Save: 0.11606550216674805 Total: 0.16109919548034668
Found 1 Planes, took 0.16109919548034668 seconds
Load: 0.046820640563964844 Detection: 0.07349514961242676 Save: 0.11298966407775879 Total: 0.15981030464172363
Found 0 Planes, took 0.15981030464172363 seconds
	- plane: 0.9956409931182861
Load: 0.04413723945617676 Detection: 0.07371163368225098 Save: 0.11595463752746582 Total: 0.16009187698364258
Found 1 Planes, took 0.16009187698364258 seconds
Load: 0.03522372245788574 Detection: 0.07324957847595215 Save: 0.1103677749633789 Total: 0.14559149742126465
Found 0 Planes, took 0.14559149742126465 seconds
	- plane: 0.9991621971130371
Load: 0.04398059844970703 Detection: 0.07317948341369629 Save: 0.11632490158081055 Total: 

	- plane: 0.9998652935028076
Load: 0.043563127517700195 Detection: 0.07327032089233398 Save: 0.11562418937683105 Total: 0.15918731689453125
Found 1 Planes, took 0.15918731689453125 seconds
Load: 0.045537710189819336 Detection: 0.07246613502502441 Save: 0.11227655410766602 Total: 0.15781426429748535
Found 0 Planes, took 0.15781426429748535 seconds
Load: 0.04704713821411133 Detection: 0.07291960716247559 Save: 0.11212420463562012 Total: 0.15917134284973145
Found 0 Planes, took 0.15917134284973145 seconds
Load: 0.04232001304626465 Detection: 0.07314038276672363 Save: 0.11241936683654785 Total: 0.1547393798828125
Found 0 Planes, took 0.1547393798828125 seconds
	- plane: 0.9999123811721802
Load: 0.044084787368774414 Detection: 0.07251429557800293 Save: 0.11726617813110352 Total: 0.16135096549987793
Found 1 Planes, took 0.16135096549987793 seconds
	- plane: 0.999729573726654
Load: 0.0449824333190918 Detection: 0.07331538200378418 Save: 0.11717081069946289 Total: 0.1621532440185547
Found 1 Pl

Load: 0.04749321937561035 Detection: 0.07268810272216797 Save: 0.10966372489929199 Total: 0.15715694427490234
Found 0 Planes, took 0.15715694427490234 seconds
Load: 0.04500007629394531 Detection: 0.07268786430358887 Save: 0.1149592399597168 Total: 0.1599593162536621
Found 0 Planes, took 0.1599593162536621 seconds
Load: 0.045453548431396484 Detection: 0.07352161407470703 Save: 0.1096804141998291 Total: 0.15513396263122559
Found 0 Planes, took 0.15513396263122559 seconds
	- plane: 0.9990577101707458
Load: 0.04422807693481445 Detection: 0.07239675521850586 Save: 0.11638426780700684 Total: 0.1606123447418213
Found 1 Planes, took 0.1606123447418213 seconds
Load: 0.043744564056396484 Detection: 0.07298874855041504 Save: 0.1102747917175293 Total: 0.15401935577392578
Found 0 Planes, took 0.15401935577392578 seconds
Load: 0.0456385612487793 Detection: 0.07270193099975586 Save: 0.10909533500671387 Total: 0.15473389625549316
Found 0 Planes, took 0.15473389625549316 seconds
	- plane: 0.99962818622

...

	- plane: 0.9994871616363525
Load: 0.04311180114746094 Detection: 0.07307982444763184 Save: 0.11940407752990723 Total: 0.16251587867736816
Found 1 Planes, took 0.16251587867736816 seconds
	- plane: 0.9997116923332214
Load: 0.04515409469604492 Detection: 0.0729970932006836 Save: 0.11670041084289551 Total: 0.16185450553894043
Found 1 Planes, took 0.16185450553894043 seconds
	- plane: 0.999992847442627
Load: 0.047899484634399414 Detection: 0.07298660278320312 Save: 0.11759543418884277 Total: 0.1654949188232422
Found 1 Planes, took 0.1654949188232422 seconds
	- plane: 0.9971202611923218
Load: 0.044818878173828125 Detection: 0.07268595695495605 Save: 0.1156151294708252 Total: 0.16043400764465332
Found 1 Planes, took 0.16043400764465332 seconds
	- plane: 0.9999685287475586
Load: 0.04553413391113281 Detection: 0.07233691215515137 Save: 0.11926865577697754 Total: 0.16480278968811035
Found 1 Planes, took 0.16480278968811035 seconds
Load: 0.04344940185546875 Detection: 0.07319045066833496 Save: 

Load: 0.0431208610534668 Detection: 0.07291126251220703 Save: 0.11155152320861816 Total: 0.15467238426208496
Found 0 Planes, took 0.15467238426208496 seconds
	- plane: 0.9950762391090393
Load: 0.0440673828125 Detection: 0.07311201095581055 Save: 0.1151878833770752 Total: 0.1592552661895752
Found 1 Planes, took 0.1592552661895752 seconds
	- plane: 0.9981879591941833
Load: 0.047089338302612305 Detection: 0.07278704643249512 Save: 0.11516261100769043 Total: 0.16225194931030273
Found 1 Planes, took 0.16225194931030273 seconds
	- plane: 0.9999706745147705
Load: 0.0446319580078125 Detection: 0.07171916961669922 Save: 0.11576676368713379 Total: 0.1603987216949463
Found 1 Planes, took 0.1603987216949463 seconds
	- plane: 0.9999266862869263
Load: 0.04492640495300293 Detection: 0.0731348991394043 Save: 0.11522412300109863 Total: 0.16015052795410156
Found 1 Planes, took 0.16015052795410156 seconds
Load: 0.04535627365112305 Detection: 0.07336544990539551 Save: 0.11306357383728027 Total: 0.15841984

	- plane: 0.9995118379592896
Load: 0.04564976692199707 Detection: 0.07226133346557617 Save: 0.11344003677368164 Total: 0.1590898036956787
Found 1 Planes, took 0.1590898036956787 seconds
Load: 0.04511833190917969 Detection: 0.07262301445007324 Save: 0.1089470386505127 Total: 0.15406537055969238
Found 0 Planes, took 0.15406537055969238 seconds
	- plane: 0.9913251399993896
Load: 0.041989803314208984 Detection: 0.07265806198120117 Save: 0.11642909049987793 Total: 0.15841889381408691
Found 1 Planes, took 0.15841889381408691 seconds
	- plane: 0.6884048581123352
Load: 0.04421663284301758 Detection: 0.07189488410949707 Save: 0.11363410949707031 Total: 0.1578507423400879
Found 1 Planes, took 0.1578507423400879 seconds
	- plane: 0.9999330043792725
Load: 0.04480266571044922 Detection: 0.07208800315856934 Save: 0.11356735229492188 Total: 0.1583700180053711
Found 1 Planes, took 0.1583700180053711 seconds
	- plane: 0.9999861717224121
Load: 0.045145273208618164 Detection: 0.07258391380310059 Save: 0.

Load: 0.042513132095336914 Detection: 0.07261919975280762 Save: 0.1126091480255127 Total: 0.1551222801208496
Found 0 Planes, took 0.1551222801208496 seconds
	- plane: 0.9967346787452698
Load: 0.04467654228210449 Detection: 0.0722494125366211 Save: 0.11331319808959961 Total: 0.1579897403717041
Found 1 Planes, took 0.1579897403717041 seconds
Load: 0.045767784118652344 Detection: 0.07232666015625 Save: 0.11176037788391113 Total: 0.15752816200256348
Found 0 Planes, took 0.15752816200256348 seconds
	- plane: 0.9998743534088135
Load: 0.04627561569213867 Detection: 0.07308697700500488 Save: 0.1146090030670166 Total: 0.16088461875915527
Found 1 Planes, took 0.16088461875915527 seconds
	- plane: 0.998809814453125
Load: 0.04646730422973633 Detection: 0.07323527336120605 Save: 0.11625862121582031 Total: 0.16272592544555664
Found 1 Planes, took 0.16272592544555664 seconds
Load: 0.04428577423095703 Detection: 0.07318258285522461 Save: 0.11393022537231445 Total: 0.15821599960327148
Found 0 Planes, t

Load: 0.04641079902648926 Detection: 0.07201719284057617 Save: 0.11146044731140137 Total: 0.15787124633789062
Found 0 Planes, took 0.15787124633789062 seconds
Load: 0.042856693267822266 Detection: 0.07360672950744629 Save: 0.10987520217895508 Total: 0.15273189544677734
Found 0 Planes, took 0.15273189544677734 seconds
Load: 0.04369544982910156 Detection: 0.07293820381164551 Save: 0.10896992683410645 Total: 0.152665376663208
Found 0 Planes, took 0.152665376663208 seconds
	- plane: 0.9995527863502502
Load: 0.043193817138671875 Detection: 0.07291054725646973 Save: 0.11498904228210449 Total: 0.15818285942077637
Found 1 Planes, took 0.15818285942077637 seconds
Load: 0.046816349029541016 Detection: 0.07279372215270996 Save: 0.1093294620513916 Total: 0.15614581108093262
Found 0 Planes, took 0.15614581108093262 seconds
Load: 0.04525160789489746 Detection: 0.07263779640197754 Save: 0.11221575736999512 Total: 0.15746736526489258
Found 0 Planes, took 0.15746736526489258 seconds
Load: 0.05004763603

# Examine the results
Here is some example code on how you could test how well the predictions match ground truth data.

## View the Results
Use the UI to examine the predictions. You can select poorly performing samples and tag them for relabeling.

In [9]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


In [11]:
print(dataset)

Name:        jsm-test-dataset
Media type:  image
Num samples: 5151
Persistent:  True
Tags:        []
Sample fields:
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    bearing:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    elevation:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    distance:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    model:            fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    manufacturer:     fiftyone.c

In [10]:
view = dataset.exists("predict_model")#.match({"relabel": {"$exists": False, "$eq": None}})
session = fo.launch_app(view, auto=False)
print(view)
#session.view = view

ValueError: `dataset` must be a <class 'fiftyone.core.dataset.Dataset'> or None; found <class 'fiftyone.core.view.DatasetView'>

### Select Samples
Select poorly performing samples in the UI and then run to code below to tag the selected samples for relabeling.

In [59]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)
print(selected_view)
for sample in selected_view:
    sample.tags.append("relabel")
    sample.save() 

Dataset:        plane-dataset
Media type:     image
Num samples:    1
Tags:           ['plane']
Sample fields:
    filepath:    fiftyone.core.fields.StringField
    tags:        fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    icao24:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    plane:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    plane_spot:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    labelbox_id: fiftyone.core.fields.StringField
    plane-box:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    plane_box:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    planebox:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Select(sample_ids=['60147b2d

ValueError: Failed to load sample from the database. This is likely due to an invalid stage in the DatasetView

In [31]:
predict_model_view = dataset.exists(field_name)
total=0
top3_total=0
for sample in predict_model_view:
    top_detect = sample["predict_model"].detections[0]
    bb_area = top_detect["bounding_box"][2] * top_detect["bounding_box"][3]
    
    if sample["norm_model"].label==top_detect["label"]:
        match="Match"
        top3_match="Top3 Match"
        total = total+1
        top3_total=top3_total+1
        found=True
        top3_found=True
    else:
        match="!NO Match!"
        top3_match="!NO TOP3 Match!"
        found=False
        top3_found=False
        for i,guess in enumerate(sample["predict_model"].detections):
            if i>3:
                break
            if sample["norm_model"].label==guess["label"]:
                top3_match="Top3 Match"
                top3_found=True
                top3_total=top3_total+1
                break
                
    #print("{}\t{}\t\t{}\t\t{}".format(bb_area,sample["norm_model"].label,match,top3_match))
    print("{}, {}, {}, {}".format(bb_area,sample["norm_model"].label,found,top3_found))
print("{}\n{}\n\n{}\n{}".format(total,100-total,top3_total,100-top3_total))

0.003363115119441318, A321, False, False
0.004131421679733904, A321, False, True
0.00306788170649952, 757-200, False, False
0.003407978022078595, 757-200, False, False
0.004803034913855697, A330, False, True
0.002368545882305284, ERJ-170, True, True
0.0023542858515170906, ERJ-170, True, True
0.002527645226944486, ERJ-170, False, True
0.0012630497521257666, Learjet 45/60, False, True
0.001340467587462868, CRJ700, False, False
0.0031371196698231074, 737-900, False, False
0.002852709476428572, 737-900, False, False
0.0024098699581678318, 737-900, False, False
0.0013388308064037346, Cessna Jet, True, True
0.002016396768755868, A320, False, True
0.0026610918499443414, 757-200, False, False
0.0029156908491483335, 757-200, False, False
0.002778857595231443, 757-200, False, False
0.0032397564838007042, 757-200, False, True
0.00216837580980922, A321, False, True
0.002636617286100318, A321, False, False
0.004325663871298957, 787-800, False, True
0.0020077339682273987, A320, True, True
0.00236754